# 🚁 Robust Human Detection in UAV Imagery
## Using HIT-UAV Infrared Thermal Dataset

**Features:**
- HIT-UAV thermal dataset ingestion with COCO format conversion
- Scene filtering to focus on SAR-relevant imagery
- Realistic augmentations: snow, smoke/fire, thermal artifacts
- Transfer learning with Faster R-CNN (or YOLOv8)
- Robust evaluation on clean vs perturbed data

**Runtime:** ~15-20 min on Colab GPU (T4)

---

## Cell 1: Environment & Setup

In [ ]:
# =============================================================================
# CELL 1: ENVIRONMENT SETUP
# Install packages and mount Drive for persistent storage
# =============================================================================

import subprocess
import sys

def install_packages():
    """Install required packages for UAV detection pipeline."""
    packages = [
        'torch',
        'torchvision',
        'albumentations>=1.3.0',
        'pycocotools',
        'opencv-python-headless',
        'matplotlib',
        'numpy',
        'Pillow',
        'tqdm',
        'gdown',  # For Google Drive downloads
        'scipy',
    ]
    
    for pkg in packages:
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])
        except:
            print(f"Warning: Could not install {pkg}")
    
    print("✓ Packages installed successfully")

# Run installation
install_packages()

# Mount Google Drive (for Colab)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    DRIVE_ROOT = '/content/drive/MyDrive/uav_detection'
    IN_COLAB = True
    print(f"✓ Google Drive mounted at {DRIVE_ROOT}")
except ImportError:
    # Running locally
    DRIVE_ROOT = './uav_detection_cache'
    IN_COLAB = False
    print(f"✓ Running locally, cache dir: {DRIVE_ROOT}")

import os
os.makedirs(DRIVE_ROOT, exist_ok=True)
os.makedirs(f"{DRIVE_ROOT}/data", exist_ok=True)
os.makedirs(f"{DRIVE_ROOT}/checkpoints", exist_ok=True)
os.makedirs(f"{DRIVE_ROOT}/outputs", exist_ok=True)

print(f"✓ Directory structure created under {DRIVE_ROOT}")

## Cell 2: Imports and Configuration

In [ ]:
# =============================================================================
# CELL 2: IMPORTS AND CONFIGURATION
# =============================================================================

import torch
import torchvision
from torchvision import transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import json
import os
import shutil
from pathlib import Path
from tqdm.auto import tqdm
import random
from typing import Dict, List, Tuple, Optional
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Configuration
class Config:
    """Central configuration for the pipeline."""
    # Paths
    DATA_ROOT = f"{DRIVE_ROOT}/data/hit_uav"
    CURATED_ROOT = f"{DRIVE_ROOT}/data/curated"
    CHECKPOINT_DIR = f"{DRIVE_ROOT}/checkpoints"
    OUTPUT_DIR = f"{DRIVE_ROOT}/outputs"
    
    # Image settings
    IMG_SIZE = 512  # Resize for fast iteration
    
    # Training settings
    BATCH_SIZE = 4
    NUM_EPOCHS = 5  # Quick training for demo
    LR = 0.001
    MOMENTUM = 0.9
    WEIGHT_DECAY = 0.0005
    
    # Detection settings
    NUM_CLASSES = 2  # background + person
    IOU_THRESHOLD = 0.5
    CONF_THRESHOLD = 0.5
    
    # Device
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Random seed
    SEED = 42

# Set seeds for reproducibility
torch.manual_seed(Config.SEED)
np.random.seed(Config.SEED)
random.seed(Config.SEED)

print(f"✓ Configuration loaded")
print(f"  Device: {Config.DEVICE}")
print(f"  Image size: {Config.IMG_SIZE}x{Config.IMG_SIZE}")
print(f"  Epochs: {Config.NUM_EPOCHS}, Batch size: {Config.BATCH_SIZE}")

## Cell 3: Data Ingestion - HIT-UAV Dataset

In [ ]:
# =============================================================================
# CELL 3: DATA INGESTION
# Download HIT-UAV dataset, with fallbacks to VisDrone or synthetic data
# =============================================================================

import urllib.request
import zipfile
import tarfile

class DatasetDownloader:
    """Handles dataset downloading with multiple fallback options."""
    
    # HIT-UAV GitHub repo contains annotations; images via Google Drive
    HIT_UAV_GITHUB = "https://github.com/suojiashun/HIT-UAV-Infrared-Thermal-Dataset"
    HIT_UAV_GDRIVE_ID = "1hLaJNqLdoOjjnl_-v1jQzNdIuLQyMhZB"  # Sample subset
    
    # Fallback: VisDrone (RGB UAV dataset)
    VISDRONE_URL = "https://github.com/VisDrone/VisDrone-Dataset"
    
    def __init__(self, data_root: str):
        self.data_root = Path(data_root)
        self.data_root.mkdir(parents=True, exist_ok=True)
        
    def download_hit_uav(self) -> bool:
        """Attempt to download HIT-UAV dataset."""
        print("Attempting to download HIT-UAV dataset...")
        
        try:
            import gdown
            
            # Check if already downloaded
            images_dir = self.data_root / "images"
            if images_dir.exists() and len(list(images_dir.glob("*.jpg"))) > 50:
                print("✓ HIT-UAV already cached in Drive")
                return True
            
            # Try downloading sample subset from Google Drive
            zip_path = self.data_root / "hit_uav_sample.zip"
            
            # Using a public sample or the main dataset
            gdown.download(
                f"https://drive.google.com/uc?id={self.HIT_UAV_GDRIVE_ID}",
                str(zip_path),
                quiet=False
            )
            
            # Extract
            with zipfile.ZipFile(zip_path, 'r') as zf:
                zf.extractall(self.data_root)
            
            zip_path.unlink()  # Remove zip
            print("✓ HIT-UAV downloaded and extracted")
            return True
            
        except Exception as e:
            print(f"  HIT-UAV download failed: {e}")
            return False
    
    def create_synthetic_thermal_dataset(self, num_images: int = 200) -> bool:
        """
        Create synthetic thermal-like images with person annotations.
        Fallback when real dataset unavailable.
        """
        print("Creating synthetic thermal dataset for demo...")
        
        images_dir = self.data_root / "images"
        images_dir.mkdir(exist_ok=True)
        
        annotations = {
            "images": [],
            "annotations": [],
            "categories": [{"id": 1, "name": "person", "supercategory": "human"}]
        }
        
        ann_id = 1
        
        for img_id in tqdm(range(1, num_images + 1), desc="Generating images"):
            # Create thermal-like background (grayscale with noise)
            h, w = Config.IMG_SIZE, Config.IMG_SIZE
            
            # Random background intensity (simulating different thermal scenes)
            base_temp = np.random.randint(40, 120)
            img = np.ones((h, w), dtype=np.uint8) * base_temp
            
            # Add terrain-like noise
            noise = np.random.normal(0, 15, (h, w))
            img = np.clip(img + noise, 0, 255).astype(np.uint8)
            
            # Add some structure (trees, buildings as blobs)
            num_structures = np.random.randint(2, 6)
            for _ in range(num_structures):
                cx, cy = np.random.randint(50, w-50), np.random.randint(50, h-50)
                radius = np.random.randint(20, 80)
                temp = np.random.randint(60, 140)
                cv2.circle(img, (cx, cy), radius, int(temp), -1)
            
            # Apply Gaussian blur for thermal camera effect
            img = cv2.GaussianBlur(img, (5, 5), 0)
            
            # Add 1-4 "person" hot spots
            num_persons = np.random.randint(1, 5)
            persons = []
            
            for _ in range(num_persons):
                # Person size varies with simulated distance
                pw = np.random.randint(15, 50)
                ph = int(pw * np.random.uniform(1.8, 2.5))  # Person aspect ratio
                
                # Random position
                px = np.random.randint(10, w - pw - 10)
                py = np.random.randint(10, h - ph - 10)
                
                # Check overlap with existing persons
                overlap = False
                for (ex, ey, ew, eh) in persons:
                    if not (px + pw < ex or px > ex + ew or py + ph < ey or py > ey + eh):
                        overlap = True
                        break
                
                if overlap:
                    continue
                    
                persons.append((px, py, pw, ph))
                
                # Draw person as hot ellipse (brighter = warmer body)
                body_temp = np.random.randint(180, 255)
                cv2.ellipse(img, (px + pw//2, py + ph//2), (pw//2, ph//2), 
                           0, 0, 360, int(body_temp), -1)
                
                # Add head (smaller, slightly brighter)
                head_r = pw // 4
                cv2.circle(img, (px + pw//2, py + head_r + 2), head_r, 
                          int(min(body_temp + 20, 255)), -1)
                
                # Add annotation
                annotations["annotations"].append({
                    "id": ann_id,
                    "image_id": img_id,
                    "category_id": 1,
                    "bbox": [px, py, pw, ph],
                    "area": pw * ph,
                    "iscrowd": 0
                })
                ann_id += 1
            
            # Final blur and noise
            img = cv2.GaussianBlur(img, (3, 3), 0)
            sensor_noise = np.random.normal(0, 3, (h, w))
            img = np.clip(img + sensor_noise, 0, 255).astype(np.uint8)
            
            # Save image
            filename = f"thermal_{img_id:04d}.jpg"
            # Convert to 3-channel for consistency
            img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            cv2.imwrite(str(images_dir / filename), img_rgb)
            
            # Add image info
            annotations["images"].append({
                "id": img_id,
                "file_name": filename,
                "width": w,
                "height": h
            })
        
        # Save annotations
        ann_path = self.data_root / "annotations.json"
        with open(ann_path, 'w') as f:
            json.dump(annotations, f)
        
        print(f"✓ Created {num_images} synthetic thermal images")
        print(f"  Total person annotations: {ann_id - 1}")
        return True
    
    def load_or_download(self) -> Tuple[Path, Path]:
        """Main entry: load dataset or download with fallbacks."""
        
        images_dir = self.data_root / "images"
        ann_path = self.data_root / "annotations.json"
        
        # Check cache first
        if images_dir.exists() and ann_path.exists():
            num_images = len(list(images_dir.glob("*.jpg")))
            if num_images >= 50:
                print(f"✓ Using cached dataset ({num_images} images)")
                return images_dir, ann_path
        
        # Try downloading HIT-UAV
        if self.download_hit_uav():
            # Convert to COCO format if needed
            self._convert_to_coco_format()
            return images_dir, ann_path
        
        # Fallback: create synthetic dataset
        print("Using synthetic dataset as fallback...")
        self.create_synthetic_thermal_dataset(num_images=250)
        return images_dir, ann_path
    
    def _convert_to_coco_format(self):
        """Convert HIT-UAV annotations to COCO format (person class only)."""
        # HIT-UAV uses YOLO format or custom XML
        # This handles conversion to standard COCO JSON
        
        ann_path = self.data_root / "annotations.json"
        if ann_path.exists():
            return  # Already converted
        
        images_dir = self.data_root / "images"
        labels_dir = self.data_root / "labels"  # YOLO format
        
        annotations = {
            "images": [],
            "annotations": [],
            "categories": [{"id": 1, "name": "person", "supercategory": "human"}]
        }
        
        ann_id = 1
        
        # HIT-UAV class mapping: 0=Person, 1=Car, 2=Bicycle, etc.
        PERSON_CLASS = 0
        
        image_files = sorted(images_dir.glob("*.jpg")) + sorted(images_dir.glob("*.png"))
        
        for img_id, img_path in enumerate(image_files, 1):
            # Read image dimensions
            img = cv2.imread(str(img_path))
            if img is None:
                continue
            h, w = img.shape[:2]
            
            annotations["images"].append({
                "id": img_id,
                "file_name": img_path.name,
                "width": w,
                "height": h
            })
            
            # Look for YOLO-format label file
            label_path = labels_dir / f"{img_path.stem}.txt"
            if label_path.exists():
                with open(label_path, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) >= 5:
                            cls_id = int(parts[0])
                            if cls_id == PERSON_CLASS:
                                # YOLO format: class cx cy width height (normalized)
                                cx, cy, bw, bh = map(float, parts[1:5])
                                
                                # Convert to COCO format (x, y, width, height in pixels)
                                x = (cx - bw/2) * w
                                y = (cy - bh/2) * h
                                box_w = bw * w
                                box_h = bh * h
                                
                                annotations["annotations"].append({
                                    "id": ann_id,
                                    "image_id": img_id,
                                    "category_id": 1,
                                    "bbox": [x, y, box_w, box_h],
                                    "area": box_w * box_h,
                                    "iscrowd": 0
                                })
                                ann_id += 1
        
        with open(ann_path, 'w') as f:
            json.dump(annotations, f)
        
        print(f"✓ Converted to COCO format: {len(annotations['images'])} images, {ann_id-1} person annotations")


# Run data ingestion
downloader = DatasetDownloader(Config.DATA_ROOT)
IMAGES_DIR, ANNOTATIONS_PATH = downloader.load_or_download()

print(f"\n✓ Dataset ready at:")
print(f"  Images: {IMAGES_DIR}")
print(f"  Annotations: {ANNOTATIONS_PATH}")

## Cell 4: Scene Filtering & Curation

In [ ]:
# =============================================================================
# CELL 4: FILTERING & CURATION
# Filter out urban/street scenes, create curated SAR-focused dataset
# =============================================================================

class SceneFilter:
    """Filter images to focus on SAR-relevant scenes (wilderness, disaster zones)."""
    
    # Keywords suggesting urban/street scenes to exclude
    URBAN_KEYWORDS = ['street', 'road', 'traffic', 'parking', 'downtown', 
                      'intersection', 'highway', 'urban', 'city']
    
    # Keywords suggesting SAR-relevant scenes to keep
    SAR_KEYWORDS = ['field', 'forest', 'mountain', 'wilderness', 'rural',
                    'search', 'rescue', 'thermal', 'night']
    
    def __init__(self, ann_path: Path, images_dir: Path):
        with open(ann_path, 'r') as f:
            self.annotations = json.load(f)
        self.images_dir = images_dir
        
    def filter_by_filename(self, filename: str) -> bool:
        """Heuristic filtering based on filename."""
        fname_lower = filename.lower()
        
        # Reject if contains urban keywords
        for kw in self.URBAN_KEYWORDS:
            if kw in fname_lower:
                return False
        
        # Accept all synthetic images
        if 'thermal_' in fname_lower or 'synthetic' in fname_lower:
            return True
            
        return True  # Accept by default
    
    def filter_by_image_content(self, img: np.ndarray) -> bool:
        """
        Simple content-based filtering using image statistics.
        Urban scenes often have more structured patterns (buildings, roads).
        """
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img
        
        # Edge density: urban scenes tend to have more edges
        edges = cv2.Canny(gray, 50, 150)
        edge_density = np.sum(edges > 0) / edges.size
        
        # High edge density might indicate urban scene
        if edge_density > 0.15:
            return False
        
        # Check for horizontal/vertical line dominance (roads, buildings)
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50, 
                                minLineLength=50, maxLineGap=10)
        if lines is not None and len(lines) > 20:
            return False  # Too many straight lines
            
        return True
    
    def create_curated_dataset(
        self, 
        output_dir: Path,
        target_size: int = 512,
        max_images: int = None,
        use_content_filter: bool = False
    ) -> Tuple[Path, Path]:
        """
        Create filtered dataset with resized images.
        
        Args:
            output_dir: Output directory
            target_size: Resize images to this size
            max_images: Limit number of images (for quick testing)
            use_content_filter: Enable slow content-based filtering
        """
        output_dir = Path(output_dir)
        images_out = output_dir / "images"
        images_out.mkdir(parents=True, exist_ok=True)
        
        # Build image id to annotations mapping
        img_to_anns = defaultdict(list)
        for ann in self.annotations['annotations']:
            img_to_anns[ann['image_id']].append(ann)
        
        filtered_images = []
        filtered_annotations = []
        new_ann_id = 1
        
        images_list = self.annotations['images']
        if max_images:
            images_list = images_list[:max_images]
        
        print(f"Filtering {len(images_list)} images...")
        
        for img_info in tqdm(images_list, desc="Curating"):
            filename = img_info['file_name']
            img_path = self.images_dir / filename
            
            # Skip if no file
            if not img_path.exists():
                continue
            
            # Skip if no person annotations
            anns = img_to_anns.get(img_info['id'], [])
            if len(anns) == 0:
                continue
            
            # Filename heuristic filter
            if not self.filter_by_filename(filename):
                continue
            
            # Load image
            img = cv2.imread(str(img_path))
            if img is None:
                continue
            
            # Content-based filter (optional, slower)
            if use_content_filter and not self.filter_by_image_content(img):
                continue
            
            # Resize image
            orig_h, orig_w = img.shape[:2]
            scale_x = target_size / orig_w
            scale_y = target_size / orig_h
            
            img_resized = cv2.resize(img, (target_size, target_size))
            
            # Save resized image
            new_filename = f"curated_{len(filtered_images):04d}.jpg"
            cv2.imwrite(str(images_out / new_filename), img_resized)
            
            new_img_id = len(filtered_images) + 1
            filtered_images.append({
                "id": new_img_id,
                "file_name": new_filename,
                "width": target_size,
                "height": target_size,
                "original_file": filename
            })
            
            # Scale annotations
            for ann in anns:
                x, y, w, h = ann['bbox']
                scaled_bbox = [
                    x * scale_x,
                    y * scale_y,
                    w * scale_x,
                    h * scale_y
                ]
                
                # Skip tiny boxes after scaling
                if scaled_bbox[2] < 5 or scaled_bbox[3] < 5:
                    continue
                
                filtered_annotations.append({
                    "id": new_ann_id,
                    "image_id": new_img_id,
                    "category_id": 1,
                    "bbox": scaled_bbox,
                    "area": scaled_bbox[2] * scaled_bbox[3],
                    "iscrowd": 0
                })
                new_ann_id += 1
        
        # Save filtered annotations
        filtered_coco = {
            "images": filtered_images,
            "annotations": filtered_annotations,
            "categories": [{"id": 1, "name": "person", "supercategory": "human"}]
        }
        
        ann_out = output_dir / "instances_filtered.json"
        with open(ann_out, 'w') as f:
            json.dump(filtered_coco, f, indent=2)
        
        print(f"\n✓ Curated dataset created:")
        print(f"  Images: {len(filtered_images)}")
        print(f"  Person annotations: {len(filtered_annotations)}")
        print(f"  Saved to: {output_dir}")
        
        return images_out, ann_out


# Create curated dataset
scene_filter = SceneFilter(ANNOTATIONS_PATH, IMAGES_DIR)
CURATED_IMAGES, CURATED_ANNOTATIONS = scene_filter.create_curated_dataset(
    output_dir=Path(Config.CURATED_ROOT),
    target_size=Config.IMG_SIZE,
    use_content_filter=False  # Set True for stricter filtering (slower)
)

## Cell 5: SAR Augmentations (Snow, Smoke/Fire, Thermal)

In [ ]:
# =============================================================================
# CELL 5: SAR-SPECIFIC AUGMENTATIONS
# Realistic augmentations for search-and-rescue conditions
# =============================================================================

class SARaugmentations:
    """Realistic augmentations for SAR drone imagery."""
    
    @staticmethod
    def generate_perlin_noise(shape: Tuple[int, int], scale: float = 100.0) -> np.ndarray:
        """
        Generate Perlin-like noise using multiple octaves of Gaussian noise.
        Used for natural-looking snow and smoke patterns.
        """
        h, w = shape
        noise = np.zeros((h, w), dtype=np.float32)
        
        # Multiple octaves for natural appearance
        for octave in range(4):
            freq = 2 ** octave
            amplitude = 1.0 / freq
            
            # Generate low-res noise and upscale
            small_h = max(2, int(h / (scale / freq)))
            small_w = max(2, int(w / (scale / freq)))
            
            small_noise = np.random.randn(small_h, small_w).astype(np.float32)
            upscaled = cv2.resize(small_noise, (w, h), interpolation=cv2.INTER_CUBIC)
            
            noise += amplitude * upscaled
        
        # Normalize to [0, 1]
        noise = (noise - noise.min()) / (noise.max() - noise.min() + 1e-8)
        return noise
    
    @staticmethod
    def apply_snow(
        img: np.ndarray,
        intensity: float = 0.5,
        flake_size: float = 2.0,
        contrast_reduction: float = 0.3
    ) -> np.ndarray:
        """
        Apply realistic snow effect.
        
        Args:
            img: Input image (BGR or grayscale)
            intensity: Overall snow intensity [0-1]
            flake_size: Size of snow particles
            contrast_reduction: How much to reduce contrast [0-1]
        """
        h, w = img.shape[:2]
        is_color = len(img.shape) == 3
        
        # Generate snow noise pattern
        snow_noise = SARaugmentations.generate_perlin_noise((h, w), scale=50.0)
        
        # Add fine grain for individual flakes
        fine_noise = np.random.rand(h, w).astype(np.float32)
        fine_noise = cv2.GaussianBlur(fine_noise, (int(flake_size*2+1), int(flake_size*2+1)), 0)
        
        # Combine noises
        snow_layer = 0.6 * snow_noise + 0.4 * fine_noise
        snow_layer = np.clip(snow_layer * intensity * 255, 0, 255).astype(np.uint8)
        
        if is_color:
            snow_layer = cv2.cvtColor(snow_layer, cv2.COLOR_GRAY2BGR)
        
        # Reduce contrast (simulates atmospheric haze during snow)
        img_float = img.astype(np.float32)
        mean_val = np.mean(img_float)
        img_reduced = (1 - contrast_reduction) * img_float + contrast_reduction * mean_val
        
        # Alpha blend snow
        alpha = intensity * 0.7
        result = (1 - alpha) * img_reduced + alpha * snow_layer.astype(np.float32)
        
        return np.clip(result, 0, 255).astype(np.uint8)
    
    @staticmethod
    def apply_smoke_fire(
        img: np.ndarray,
        smoke_intensity: float = 0.4,
        fire_intensity: float = 0.3,
        fire_location: str = 'random'  # 'random', 'bottom', 'corner'
    ) -> np.ndarray:
        """
        Apply smoke and fire effects.
        
        Args:
            img: Input image
            smoke_intensity: Opacity of smoke [0-1]
            fire_intensity: Intensity of fire glow [0-1]
            fire_location: Where to place fire hotspot
        """
        h, w = img.shape[:2]
        is_color = len(img.shape) == 3
        
        if not is_color:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        
        result = img.astype(np.float32)
        
        # Generate fractal smoke pattern
        smoke_noise = SARaugmentations.generate_perlin_noise((h, w), scale=80.0)
        
        # Smoke rises - gradient mask
        gradient = np.linspace(1.0, 0.3, h).reshape(-1, 1)
        gradient = np.tile(gradient, (1, w))
        smoke_mask = smoke_noise * gradient
        
        # Apply smoke (gray overlay with blur)
        smoke_color = np.array([180, 180, 180], dtype=np.float32)  # Gray smoke
        smoke_layer = np.ones((h, w, 3), dtype=np.float32) * smoke_color
        smoke_layer = cv2.GaussianBlur(smoke_layer, (21, 21), 0)
        
        smoke_alpha = smoke_mask[..., np.newaxis] * smoke_intensity
        result = result * (1 - smoke_alpha) + smoke_layer * smoke_alpha
        
        # Add fire glow
        if fire_intensity > 0:
            # Fire hotspot location
            if fire_location == 'random':
                fx, fy = np.random.randint(w//4, 3*w//4), np.random.randint(h//2, h)
            elif fire_location == 'bottom':
                fx, fy = w//2, int(h * 0.8)
            else:  # corner
                fx, fy = int(w * 0.2), int(h * 0.8)
            
            # Create fire glow gradient
            y_coords, x_coords = np.ogrid[:h, :w]
            dist = np.sqrt((x_coords - fx)**2 + (y_coords - fy)**2)
            fire_radius = min(h, w) // 3
            fire_mask = np.clip(1 - dist / fire_radius, 0, 1) ** 2
            
            # Fire colors (orange-red in BGR)
            fire_color = np.array([30, 100, 255], dtype=np.float32)  # Orange
            fire_layer = np.ones((h, w, 3), dtype=np.float32) * fire_color
            
            fire_alpha = fire_mask[..., np.newaxis] * fire_intensity
            result = result * (1 - fire_alpha) + fire_layer * fire_alpha
        
        return np.clip(result, 0, 255).astype(np.uint8)
    
    @staticmethod
    def apply_thermal_artifacts(
        img: np.ndarray,
        intensity_scale: float = 1.0,
        sensor_noise: float = 0.05,
        local_saturation: float = 0.1
    ) -> np.ndarray:
        """
        Apply thermal camera artifacts.
        
        Args:
            img: Input image
            intensity_scale: Global intensity scaling factor
            sensor_noise: Amount of sensor noise [0-1]
            local_saturation: Probability of local saturation spots
        """
        h, w = img.shape[:2]
        
        # Convert to grayscale for thermal processing
        if len(img.shape) == 3:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        else:
            gray = img.copy()
        
        result = gray.astype(np.float32)
        
        # Intensity scaling (simulates different thermal ranges)
        result = result * intensity_scale
        
        # Add sensor noise (thermal cameras have characteristic noise)
        noise = np.random.normal(0, sensor_noise * 255, (h, w)).astype(np.float32)
        
        # Temporal noise pattern (horizontal lines common in thermal)
        if np.random.rand() < 0.3:
            line_noise = np.random.normal(0, sensor_noise * 100, (h, 1))
            line_noise = np.tile(line_noise, (1, w))
            noise += line_noise
        
        result += noise
        
        # Local saturation spots (hot/cold spots)
        if np.random.rand() < local_saturation:
            num_spots = np.random.randint(1, 4)
            for _ in range(num_spots):
                sx, sy = np.random.randint(0, w), np.random.randint(0, h)
                radius = np.random.randint(5, 30)
                value = 255 if np.random.rand() > 0.5 else 0  # Hot or cold
                cv2.circle(result, (sx, sy), radius, value, -1)
        
        result = np.clip(result, 0, 255).astype(np.uint8)
        
        # Convert back to 3-channel
        return cv2.cvtColor(result, cv2.COLOR_GRAY2BGR)
    
    @staticmethod
    def apply_random_sar_augmentation(
        img: np.ndarray,
        augmentation_type: str = 'random'
    ) -> Tuple[np.ndarray, str]:
        """
        Apply random SAR augmentation.
        
        Args:
            img: Input image
            augmentation_type: 'snow', 'fire', 'thermal', or 'random'
        
        Returns:
            Tuple of (augmented image, augmentation name)
        """
        if augmentation_type == 'random':
            augmentation_type = np.random.choice(['snow', 'fire', 'thermal', 'clean'])
        
        if augmentation_type == 'snow':
            intensity = np.random.uniform(0.3, 0.7)
            return SARaugmentations.apply_snow(img, intensity=intensity), 'snow'
        
        elif augmentation_type == 'fire':
            smoke = np.random.uniform(0.2, 0.5)
            fire = np.random.uniform(0.2, 0.5)
            return SARaugmentations.apply_smoke_fire(img, smoke, fire), 'fire'
        
        elif augmentation_type == 'thermal':
            scale = np.random.uniform(0.8, 1.2)
            noise = np.random.uniform(0.02, 0.1)
            return SARaugmentations.apply_thermal_artifacts(img, scale, noise), 'thermal'
        
        else:
            return img, 'clean'


def create_perturbed_test_set(
    images_dir: Path,
    annotations_path: Path,
    output_dir: Path,
    perturbation_types: List[str] = ['snow', 'fire']
) -> Path:
    """
    Create a perturbed version of the test set for robustness evaluation.
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    with open(annotations_path, 'r') as f:
        annotations = json.load(f)
    
    print(f"Creating perturbed test set with: {perturbation_types}")
    
    for img_info in tqdm(annotations['images'], desc="Perturbing"):
        img_path = images_dir / img_info['file_name']
        img = cv2.imread(str(img_path))
        
        if img is None:
            continue
        
        # Apply random perturbation
        aug_type = np.random.choice(perturbation_types)
        perturbed, _ = SARaugmentations.apply_random_sar_augmentation(img, aug_type)
        
        # Save
        cv2.imwrite(str(output_dir / img_info['file_name']), perturbed)
    
    # Copy annotations (same boxes, different images)
    shutil.copy(annotations_path, output_dir / "instances_perturbed.json")
    
    print(f"✓ Perturbed test set saved to {output_dir}")
    return output_dir


# Demonstrate augmentations
print("\n" + "="*50)
print("SAR AUGMENTATION EXAMPLES")
print("="*50)

# Load a sample image
sample_images = list(CURATED_IMAGES.glob("*.jpg"))[:1]
if sample_images:
    sample_img = cv2.imread(str(sample_images[0]))
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    
    axes[0, 0].imshow(cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB))
    axes[0, 0].set_title('Original')
    axes[0, 0].axis('off')
    
    snow_img = SARaugmentations.apply_snow(sample_img, intensity=0.5)
    axes[0, 1].imshow(cv2.cvtColor(snow_img, cv2.COLOR_BGR2RGB))
    axes[0, 1].set_title('Snow Effect')
    axes[0, 1].axis('off')
    
    fire_img = SARaugmentations.apply_smoke_fire(sample_img, smoke_intensity=0.4, fire_intensity=0.4)
    axes[1, 0].imshow(cv2.cvtColor(fire_img, cv2.COLOR_BGR2RGB))
    axes[1, 0].set_title('Smoke/Fire Effect')
    axes[1, 0].axis('off')
    
    thermal_img = SARaugmentations.apply_thermal_artifacts(sample_img, intensity_scale=1.1, sensor_noise=0.08)
    axes[1, 1].imshow(cv2.cvtColor(thermal_img, cv2.COLOR_BGR2RGB))
    axes[1, 1].set_title('Thermal Artifacts')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.savefig(f"{Config.OUTPUT_DIR}/augmentation_examples.png", dpi=150)
    plt.show()
    print(f"✓ Augmentation examples saved to {Config.OUTPUT_DIR}/augmentation_examples.png")

## Cell 6: Dataset & DataLoader

In [ ]:
# =============================================================================
# CELL 6: PYTORCH DATASET AND DATALOADER
# Custom dataset with on-the-fly augmentation support
# =============================================================================

from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2


class UAVPersonDataset(Dataset):
    """
    PyTorch Dataset for UAV person detection.
    
    Supports:
    - COCO-format annotations
    - On-the-fly SAR augmentations
    - Standard albumentations transforms
    """
    
    def __init__(
        self,
        images_dir: Path,
        annotations_path: Path,
        transform=None,
        apply_sar_augmentation: bool = False,
        sar_aug_prob: float = 0.5
    ):
        self.images_dir = Path(images_dir)
        self.transform = transform
        self.apply_sar_augmentation = apply_sar_augmentation
        self.sar_aug_prob = sar_aug_prob
        
        # Load COCO annotations
        with open(annotations_path, 'r') as f:
            coco = json.load(f)
        
        self.images = {img['id']: img for img in coco['images']}
        
        # Group annotations by image
        self.img_to_anns = defaultdict(list)
        for ann in coco['annotations']:
            self.img_to_anns[ann['image_id']].append(ann)
        
        # Only keep images with annotations
        self.img_ids = [img_id for img_id in self.images.keys() 
                        if len(self.img_to_anns[img_id]) > 0]
        
    def __len__(self):
        return len(self.img_ids)
    
    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_info = self.images[img_id]
        
        # Load image
        img_path = self.images_dir / img_info['file_name']
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Apply SAR augmentation
        if self.apply_sar_augmentation and np.random.rand() < self.sar_aug_prob:
            img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            img_aug, _ = SARaugmentations.apply_random_sar_augmentation(img_bgr)
            img = cv2.cvtColor(img_aug, cv2.COLOR_BGR2RGB)
        
        # Get annotations
        anns = self.img_to_anns[img_id]
        
        boxes = []
        labels = []
        areas = []
        
        for ann in anns:
            x, y, w, h = ann['bbox']
            # Convert to [x1, y1, x2, y2] format
            boxes.append([x, y, x + w, y + h])
            labels.append(ann['category_id'])
            areas.append(ann['area'])
        
        # Apply transforms
        if self.transform:
            transformed = self.transform(
                image=img,
                bboxes=boxes,
                labels=labels
            )
            img = transformed['image']
            boxes = transformed['bboxes']
            labels = transformed['labels']
        else:
            # Default: just convert to tensor
            img = torch.from_numpy(img).permute(2, 0, 1).float() / 255.0
        
        # Prepare target dict for Faster R-CNN
        if len(boxes) > 0:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)
            areas = torch.as_tensor(areas, dtype=torch.float32)
        else:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
            areas = torch.zeros((0,), dtype=torch.float32)
        
        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([img_id]),
            'area': areas,
            'iscrowd': torch.zeros((len(boxes),), dtype=torch.int64)
        }
        
        return img, target


def get_transforms(train: bool = True):
    """Get albumentations transforms for training/validation."""
    if train:
        return A.Compose([
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.3),
            A.GaussNoise(var_limit=(10, 50), p=0.2),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ], bbox_params=A.BboxParams(
            format='pascal_voc',
            label_fields=['labels'],
            min_visibility=0.3
        ))
    else:
        return A.Compose([
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ], bbox_params=A.BboxParams(
            format='pascal_voc',
            label_fields=['labels']
        ))


def collate_fn(batch):
    """Custom collate function for detection."""
    return tuple(zip(*batch))


# Create train/val split
with open(CURATED_ANNOTATIONS, 'r') as f:
    full_annotations = json.load(f)

# 80/20 train/val split
all_images = full_annotations['images']
np.random.shuffle(all_images)
split_idx = int(0.8 * len(all_images))

train_images = all_images[:split_idx]
val_images = all_images[split_idx:]

train_img_ids = set(img['id'] for img in train_images)
val_img_ids = set(img['id'] for img in val_images)

train_anns = [a for a in full_annotations['annotations'] if a['image_id'] in train_img_ids]
val_anns = [a for a in full_annotations['annotations'] if a['image_id'] in val_img_ids]

# Save split annotations
train_coco = {'images': train_images, 'annotations': train_anns, 'categories': full_annotations['categories']}
val_coco = {'images': val_images, 'annotations': val_anns, 'categories': full_annotations['categories']}

train_ann_path = Path(Config.CURATED_ROOT) / "instances_train.json"
val_ann_path = Path(Config.CURATED_ROOT) / "instances_val.json"

with open(train_ann_path, 'w') as f:
    json.dump(train_coco, f)
with open(val_ann_path, 'w') as f:
    json.dump(val_coco, f)

print(f"\n✓ Dataset split created:")
print(f"  Train: {len(train_images)} images, {len(train_anns)} annotations")
print(f"  Val: {len(val_images)} images, {len(val_anns)} annotations")

# Create datasets
train_dataset = UAVPersonDataset(
    CURATED_IMAGES, train_ann_path,
    transform=get_transforms(train=True),
    apply_sar_augmentation=True,
    sar_aug_prob=0.3
)

val_dataset = UAVPersonDataset(
    CURATED_IMAGES, val_ann_path,
    transform=get_transforms(train=False),
    apply_sar_augmentation=False
)

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=Config.BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    val_dataset,
    batch_size=Config.BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
)

print(f"\n✓ DataLoaders created:")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")

## Cell 7: Model Setup (Faster R-CNN with Transfer Learning)

In [ ]:
# =============================================================================
# CELL 7: MODEL SETUP
# Load pre-trained Faster R-CNN and adapt for person detection
# =============================================================================

# ============================================
# ALTERNATIVE: To use YOLOv8 instead, uncomment:
# from ultralytics import YOLO
# model = YOLO('yolov8n.pt')  # or yolov8s.pt
# # Fine-tune with: model.train(data='your_data.yaml', epochs=5)
# ============================================

def create_model(num_classes: int = 2, pretrained: bool = True, freeze_backbone: bool = True):
    """
    Create Faster R-CNN model adapted for person detection.
    
    Args:
        num_classes: Number of classes (including background)
        pretrained: Use pretrained COCO weights
        freeze_backbone: Freeze backbone layers initially
    """
    # Load pretrained Faster R-CNN
    if pretrained:
        weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
        model = fasterrcnn_resnet50_fpn(weights=weights)
        print("✓ Loaded pretrained Faster R-CNN (COCO weights)")
    else:
        model = fasterrcnn_resnet50_fpn(weights=None)
        print("✓ Created Faster R-CNN from scratch")
    
    # Replace classification head for our classes
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    print(f"✓ Replaced head for {num_classes} classes")
    
    # Freeze backbone layers for initial training
    if freeze_backbone:
        for name, param in model.named_parameters():
            if 'backbone' in name:
                # Freeze all except last layer of backbone
                if 'layer4' not in name and 'fpn' not in name:
                    param.requires_grad = False
        
        # Count trainable params
        trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
        total = sum(p.numel() for p in model.parameters())
        print(f"✓ Frozen early backbone layers")
        print(f"  Trainable params: {trainable:,} / {total:,} ({100*trainable/total:.1f}%)")
    
    return model


# Create model
model = create_model(
    num_classes=Config.NUM_CLASSES,
    pretrained=True,
    freeze_backbone=True
)

model.to(Config.DEVICE)
print(f"\n✓ Model moved to {Config.DEVICE}")

# ============================================
# FOR RGB DATASETS (e.g., VisDrone, Okutama):
# The model works the same way. Key differences:
# 1. No need for thermal-specific augmentations
# 2. Use standard ImageNet normalization (already set)
# 3. Consider adding color augmentations
# ============================================

## Cell 8: Training Loop

In [ ]:
# =============================================================================
# CELL 8: TRAINING LOOP
# Lightweight training with progress tracking
# =============================================================================

def train_one_epoch(model, optimizer, data_loader, device, epoch):
    """Train for one epoch."""
    model.train()
    
    total_loss = 0
    num_batches = 0
    
    pbar = tqdm(data_loader, desc=f"Epoch {epoch+1}")
    
    for images, targets in pbar:
        # Move to device
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # Skip empty batches
        if len(images) == 0:
            continue
        
        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        # Backward pass
        optimizer.zero_grad()
        losses.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        # Track loss
        total_loss += losses.item()
        num_batches += 1
        
        # Update progress bar
        pbar.set_postfix({
            'loss': f"{losses.item():.4f}",
            'avg': f"{total_loss/num_batches:.4f}"
        })
    
    return total_loss / max(num_batches, 1)


def compute_iou(box1, box2):
    """Compute IoU between two boxes [x1, y1, x2, y2]."""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / max(union_area, 1e-6)


@torch.no_grad()
def evaluate(model, data_loader, device, iou_threshold=0.5, conf_threshold=0.5):
    """
    Evaluate model and compute precision, recall, F1 @ IoU threshold.
    """
    model.eval()
    
    all_tp = 0
    all_fp = 0
    all_fn = 0
    
    for images, targets in tqdm(data_loader, desc="Evaluating"):
        images = [img.to(device) for img in images]
        
        # Get predictions
        outputs = model(images)
        
        for output, target in zip(outputs, targets):
            # Filter predictions by confidence
            pred_boxes = output['boxes'][output['scores'] > conf_threshold].cpu().numpy()
            gt_boxes = target['boxes'].cpu().numpy()
            
            # Match predictions to ground truth
            matched_gt = set()
            
            for pred_box in pred_boxes:
                best_iou = 0
                best_gt_idx = -1
                
                for gt_idx, gt_box in enumerate(gt_boxes):
                    if gt_idx in matched_gt:
                        continue
                    iou = compute_iou(pred_box, gt_box)
                    if iou > best_iou:
                        best_iou = iou
                        best_gt_idx = gt_idx
                
                if best_iou >= iou_threshold:
                    all_tp += 1
                    matched_gt.add(best_gt_idx)
                else:
                    all_fp += 1
            
            # Count missed ground truth as false negatives
            all_fn += len(gt_boxes) - len(matched_gt)
    
    # Compute metrics
    precision = all_tp / max(all_tp + all_fp, 1)
    recall = all_tp / max(all_tp + all_fn, 1)
    f1 = 2 * precision * recall / max(precision + recall, 1e-6)
    
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tp': all_tp,
        'fp': all_fp,
        'fn': all_fn
    }


# Optimizer and scheduler
optimizer = torch.optim.SGD(
    [p for p in model.parameters() if p.requires_grad],
    lr=Config.LR,
    momentum=Config.MOMENTUM,
    weight_decay=Config.WEIGHT_DECAY
)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

print("\n" + "="*60)
print("TRAINING")
print("="*60)

# Training history
history = {
    'train_loss': [],
    'val_precision': [],
    'val_recall': [],
    'val_f1': []
}

best_f1 = 0

for epoch in range(Config.NUM_EPOCHS):
    # Train
    train_loss = train_one_epoch(model, optimizer, train_loader, Config.DEVICE, epoch)
    history['train_loss'].append(train_loss)
    
    # Evaluate on clean validation set
    metrics = evaluate(model, val_loader, Config.DEVICE, 
                       iou_threshold=Config.IOU_THRESHOLD,
                       conf_threshold=Config.CONF_THRESHOLD)
    
    history['val_precision'].append(metrics['precision'])
    history['val_recall'].append(metrics['recall'])
    history['val_f1'].append(metrics['f1'])
    
    # Update scheduler
    scheduler.step()
    
    # Print metrics
    print(f"\nEpoch {epoch+1}/{Config.NUM_EPOCHS}:")
    print(f"  Train Loss: {train_loss:.4f}")
    print(f"  Val Precision: {metrics['precision']:.4f}")
    print(f"  Val Recall: {metrics['recall']:.4f}")
    print(f"  Val F1: {metrics['f1']:.4f}")
    
    # Save best model
    if metrics['f1'] > best_f1:
        best_f1 = metrics['f1']
        checkpoint_path = f"{Config.CHECKPOINT_DIR}/best_model.pth"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'f1': best_f1,
            'metrics': metrics
        }, checkpoint_path)
        print(f"  ✓ Saved best model (F1: {best_f1:.4f})")

print(f"\n✓ Training complete! Best F1: {best_f1:.4f}")
print(f"  Checkpoint saved to: {Config.CHECKPOINT_DIR}/best_model.pth")

## Cell 9: Robustness Evaluation (Perturbed Test Set)

In [ ]:
# =============================================================================
# CELL 9: ROBUSTNESS EVALUATION
# Evaluate on synthetically perturbed data (snow/fire)
# =============================================================================

print("\n" + "="*60)
print("ROBUSTNESS EVALUATION")
print("="*60)

# Create perturbed validation set
perturbed_dir = Path(Config.CURATED_ROOT) / "perturbed_val"
perturbed_images_dir = perturbed_dir / "images"
perturbed_images_dir.mkdir(parents=True, exist_ok=True)

# Load validation annotations
with open(val_ann_path, 'r') as f:
    val_annotations = json.load(f)

print(f"Creating perturbed validation images...")

for img_info in tqdm(val_annotations['images'], desc="Perturbing"):
    img_path = CURATED_IMAGES / img_info['file_name']
    img = cv2.imread(str(img_path))
    
    if img is None:
        continue
    
    # Apply random snow or fire perturbation
    aug_type = np.random.choice(['snow', 'fire'])
    perturbed, _ = SARaugmentations.apply_random_sar_augmentation(img, aug_type)
    
    # Save perturbed image
    cv2.imwrite(str(perturbed_images_dir / img_info['file_name']), perturbed)

# Save annotations for perturbed set
perturbed_ann_path = perturbed_dir / "instances_perturbed.json"
with open(perturbed_ann_path, 'w') as f:
    json.dump(val_annotations, f)

print(f"✓ Perturbed validation set created: {perturbed_dir}")

# Create perturbed dataset and loader
perturbed_dataset = UAVPersonDataset(
    perturbed_images_dir, 
    perturbed_ann_path,
    transform=get_transforms(train=False),
    apply_sar_augmentation=False
)

perturbed_loader = DataLoader(
    perturbed_dataset,
    batch_size=Config.BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
)

# Load best model
checkpoint = torch.load(f"{Config.CHECKPOINT_DIR}/best_model.pth", map_location=Config.DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"\n✓ Loaded best model from epoch {checkpoint['epoch']+1}")

# Evaluate on clean validation set
print("\nEvaluating on CLEAN validation set...")
clean_metrics = evaluate(model, val_loader, Config.DEVICE,
                         iou_threshold=Config.IOU_THRESHOLD,
                         conf_threshold=Config.CONF_THRESHOLD)

# Evaluate on perturbed validation set
print("\nEvaluating on PERTURBED validation set...")
perturbed_metrics = evaluate(model, perturbed_loader, Config.DEVICE,
                             iou_threshold=Config.IOU_THRESHOLD,
                             conf_threshold=Config.CONF_THRESHOLD)

# Print comparison
print("\n" + "="*60)
print("ROBUSTNESS COMPARISON")
print("="*60)
print(f"{'Metric':<15} {'Clean':<12} {'Perturbed':<12} {'Delta':<12}")
print("-" * 50)
print(f"{'Precision':<15} {clean_metrics['precision']:.4f}       {perturbed_metrics['precision']:.4f}       {perturbed_metrics['precision']-clean_metrics['precision']:+.4f}")
print(f"{'Recall':<15} {clean_metrics['recall']:.4f}       {perturbed_metrics['recall']:.4f}       {perturbed_metrics['recall']-clean_metrics['recall']:+.4f}")
print(f"{'F1 Score':<15} {clean_metrics['f1']:.4f}       {perturbed_metrics['f1']:.4f}       {perturbed_metrics['f1']-clean_metrics['f1']:+.4f}")
print("="*60)

robustness_drop = (clean_metrics['f1'] - perturbed_metrics['f1']) / max(clean_metrics['f1'], 1e-6) * 100
print(f"\nRobustness drop: {robustness_drop:.1f}%")

if robustness_drop < 10:
    print("✓ Model shows good robustness to SAR conditions!")
elif robustness_drop < 25:
    print("⚠ Model shows moderate sensitivity to SAR conditions")
else:
    print("⚠ Model significantly degrades under SAR conditions - consider more augmentation")

## Cell 10: Visualizations

In [ ]:
# =============================================================================
# CELL 10: VISUALIZATIONS
# Show predictions on clean vs perturbed images
# =============================================================================

def visualize_predictions(
    model,
    images_dir: Path,
    annotations_path: Path,
    num_samples: int = 4,
    conf_threshold: float = 0.5,
    title_prefix: str = ""
):
    """Visualize model predictions on sample images."""
    model.eval()
    
    with open(annotations_path, 'r') as f:
        annotations = json.load(f)
    
    # Group annotations by image
    img_to_anns = defaultdict(list)
    for ann in annotations['annotations']:
        img_to_anns[ann['image_id']].append(ann)
    
    # Sample images
    sample_images = random.sample(annotations['images'], min(num_samples, len(annotations['images'])))
    
    fig, axes = plt.subplots(2, num_samples, figsize=(4*num_samples, 8))
    if num_samples == 1:
        axes = axes.reshape(2, 1)
    
    for idx, img_info in enumerate(sample_images):
        img_path = images_dir / img_info['file_name']
        img = cv2.imread(str(img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Prepare for model
        img_tensor = torch.from_numpy(img_rgb).permute(2, 0, 1).float() / 255.0
        img_tensor = img_tensor.to(Config.DEVICE)
        
        # Get predictions
        with torch.no_grad():
            output = model([img_tensor])[0]
        
        # Ground truth
        gt_img = img_rgb.copy()
        gt_anns = img_to_anns.get(img_info['id'], [])
        for ann in gt_anns:
            x, y, w, h = map(int, ann['bbox'])
            cv2.rectangle(gt_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        axes[0, idx].imshow(gt_img)
        axes[0, idx].set_title(f"{title_prefix}GT ({len(gt_anns)} persons)")
        axes[0, idx].axis('off')
        
        # Predictions
        pred_img = img_rgb.copy()
        pred_boxes = output['boxes'][output['scores'] > conf_threshold].cpu().numpy()
        pred_scores = output['scores'][output['scores'] > conf_threshold].cpu().numpy()
        
        for box, score in zip(pred_boxes, pred_scores):
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(pred_img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(pred_img, f"{score:.2f}", (x1, y1-5), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        
        axes[1, idx].imshow(pred_img)
        axes[1, idx].set_title(f"{title_prefix}Pred ({len(pred_boxes)} detections)")
        axes[1, idx].axis('off')
    
    plt.tight_layout()
    return fig


print("\n" + "="*60)
print("VISUALIZATION: CLEAN VS PERTURBED")
print("="*60)

# Visualize clean images
print("\nClean validation set predictions:")
fig_clean = visualize_predictions(
    model, CURATED_IMAGES, val_ann_path,
    num_samples=4, conf_threshold=Config.CONF_THRESHOLD,
    title_prefix="Clean: "
)
fig_clean.savefig(f"{Config.OUTPUT_DIR}/predictions_clean.png", dpi=150)
plt.show()

# Visualize perturbed images
print("\nPerturbed validation set predictions:")
fig_perturbed = visualize_predictions(
    model, perturbed_images_dir, perturbed_ann_path,
    num_samples=4, conf_threshold=Config.CONF_THRESHOLD,
    title_prefix="Perturbed: "
)
fig_perturbed.savefig(f"{Config.OUTPUT_DIR}/predictions_perturbed.png", dpi=150)
plt.show()

# Training curves
print("\nTraining history:")
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history['train_loss'], 'b-o', label='Train Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(history['val_precision'], 'g-o', label='Precision')
axes[1].plot(history['val_recall'], 'b-s', label='Recall')
axes[1].plot(history['val_f1'], 'r-^', label='F1')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Score')
axes[1].set_title('Validation Metrics')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(f"{Config.OUTPUT_DIR}/training_history.png", dpi=150)
plt.show()

print(f"\n✓ Visualizations saved to {Config.OUTPUT_DIR}/")

## Cell 11: Save Outputs & Summary

In [ ]:
# =============================================================================
# CELL 11: SAVE OUTPUTS & SUMMARY
# Save all artifacts to Drive for persistence
# =============================================================================

print("\n" + "="*60)
print("SAVING OUTPUTS")
print("="*60)

# Save sample perturbed images
sample_perturbed_dir = Path(Config.OUTPUT_DIR) / "sample_perturbed"
sample_perturbed_dir.mkdir(exist_ok=True)

perturbed_files = list(perturbed_images_dir.glob("*.jpg"))[:10]
for f in perturbed_files:
    shutil.copy(f, sample_perturbed_dir / f.name)

print(f"✓ Sample perturbed images saved to {sample_perturbed_dir}")

# Copy filtered annotations
shutil.copy(CURATED_ANNOTATIONS, f"{Config.OUTPUT_DIR}/instances_filtered.json")
print(f"✓ Filtered annotations saved to {Config.OUTPUT_DIR}/instances_filtered.json")

# Save metrics summary
summary = {
    'config': {
        'image_size': Config.IMG_SIZE,
        'num_epochs': Config.NUM_EPOCHS,
        'batch_size': Config.BATCH_SIZE,
        'learning_rate': Config.LR
    },
    'dataset': {
        'train_images': len(train_images),
        'val_images': len(val_images),
        'train_annotations': len(train_anns),
        'val_annotations': len(val_anns)
    },
    'training_history': history,
    'best_epoch': checkpoint['epoch'] + 1,
    'clean_metrics': clean_metrics,
    'perturbed_metrics': perturbed_metrics,
    'robustness_drop_percent': robustness_drop
}

with open(f"{Config.OUTPUT_DIR}/experiment_summary.json", 'w') as f:
    json.dump(summary, f, indent=2)

print(f"✓ Experiment summary saved to {Config.OUTPUT_DIR}/experiment_summary.json")

# Final summary
print("\n" + "="*60)
print("EXPERIMENT COMPLETE")
print("="*60)
print(f"""
Dataset:
  - Training images: {len(train_images)}
  - Validation images: {len(val_images)}
  - Image size: {Config.IMG_SIZE}x{Config.IMG_SIZE}

Training:
  - Epochs: {Config.NUM_EPOCHS}
  - Best epoch: {checkpoint['epoch'] + 1}
  - Final train loss: {history['train_loss'][-1]:.4f}

Clean Validation Performance:
  - Precision: {clean_metrics['precision']:.4f}
  - Recall: {clean_metrics['recall']:.4f}
  - F1: {clean_metrics['f1']:.4f}

Perturbed Validation Performance:
  - Precision: {perturbed_metrics['precision']:.4f}
  - Recall: {perturbed_metrics['recall']:.4f}
  - F1: {perturbed_metrics['f1']:.4f}

Robustness:
  - Performance drop: {robustness_drop:.1f}%

Saved Artifacts:
  - Checkpoint: {Config.CHECKPOINT_DIR}/best_model.pth
  - Filtered annotations: {Config.OUTPUT_DIR}/instances_filtered.json
  - Visualizations: {Config.OUTPUT_DIR}/*.png
  - Summary: {Config.OUTPUT_DIR}/experiment_summary.json
""")

print("\n" + "="*60)
print("🎉 DEMO READY! All outputs saved to Google Drive.")
print("="*60)

## Cell 12: Quick Inference Demo

In [ ]:
# =============================================================================
# CELL 12: QUICK INFERENCE DEMO
# Run inference on a single image (for hackathon demo)
# =============================================================================

def run_inference(image_path: str, conf_threshold: float = 0.5):
    """
    Run inference on a single image and display results.
    
    Args:
        image_path: Path to input image
        conf_threshold: Confidence threshold for detections
    """
    # Load image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image {image_path}")
        return
    
    # Resize to model input size
    img_resized = cv2.resize(img, (Config.IMG_SIZE, Config.IMG_SIZE))
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    
    # Prepare tensor
    img_tensor = torch.from_numpy(img_rgb).permute(2, 0, 1).float() / 255.0
    img_tensor = img_tensor.to(Config.DEVICE)
    
    # Run inference
    model.eval()
    with torch.no_grad():
        output = model([img_tensor])[0]
    
    # Filter by confidence
    mask = output['scores'] > conf_threshold
    boxes = output['boxes'][mask].cpu().numpy()
    scores = output['scores'][mask].cpu().numpy()
    
    # Draw detections
    result_img = img_rgb.copy()
    for box, score in zip(boxes, scores):
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(result_img, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(result_img, f"Person {score:.2f}", (x1, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    # Display
    plt.figure(figsize=(10, 10))
    plt.imshow(result_img)
    plt.title(f"Detected {len(boxes)} person(s)")
    plt.axis('off')
    plt.show()
    
    print(f"Found {len(boxes)} person(s) with confidence > {conf_threshold}")
    return boxes, scores


# Demo: run on a sample image
sample_files = list(CURATED_IMAGES.glob("*.jpg"))
if sample_files:
    print("\n" + "="*60)
    print("QUICK INFERENCE DEMO")
    print("="*60)
    
    # Run on random sample
    sample_path = str(random.choice(sample_files))
    print(f"\nRunning inference on: {sample_path}")
    run_inference(sample_path, conf_threshold=0.5)
    
    print("\n💡 To run on your own image:")
    print("   run_inference('/path/to/your/image.jpg')")

---

## 📋 Usage Notes

### Swapping to RGB Datasets
To use RGB datasets (e.g., VisDrone, Okutama):
1. Replace the data download URL in `DatasetDownloader`
2. Adjust the annotation conversion if not in COCO format
3. Disable thermal-specific augmentations (`apply_sar_augmentation=False`)
4. Consider adding color jitter augmentations for RGB

### Swapping to YOLOv8
Replace the model setup cell with:
```python
from ultralytics import YOLO
model = YOLO('yolov8n.pt')  # or yolov8s.pt for better accuracy
model.train(
    data='your_data.yaml',
    epochs=5,
    imgsz=512,
    batch=4
)
```

### Quick 3-Minute Demo
1. Run Cells 1-3 (setup + data)
2. Skip to Cell 12 for inference on pre-trained model
3. Show augmentation examples from Cell 5

---

**Created for SAR drone hackathon prototyping**